In [ ]:
import keras
import matplotlib.pyplot as plt

from keras import layers
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 1e-3
DROPOUT_RATE = 0.1

TRAIN_SIZE = 60_000
TEST_SIZE = 10_000

INPUT_LAYER_SHAPE = 6
INPUTS_SHAPE = 784

DENCE_LAYER_NUM = 196
OUTPUTS_NUM = 10

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(TRAIN_SIZE, INPUTS_SHAPE).astype('float32') / 255
x_test = x_test.reshape(TEST_SIZE, INPUTS_SHAPE).astype('float32') / 255

In [ ]:
def custom_layer(x):
    x = keras.ops.stack([
        x[:, 0] - 0.,
        x[:, 1] - 0.2,
        x[:, 2] - 0.4,
        x[:, 3] - 0.6,
        x[:, 4] - 0.8,
        x[:, 5] - 1.
    ], axis=1)
    x = keras.ops.square(x) / 0.028
    x = -x
    return keras.ops.exp(x)

In [ ]:
input_layers = keras.Input(shape=(INPUT_LAYER_SHAPE,))
outputs = custom_layer(input_layers)
modrf = keras.Model(inputs=input_layers, outputs=outputs)

inputs = keras.Input(shape=(INPUTS_SHAPE,), name='img')
# Normalizing inputs
inpb = inputs / keras.ops.amax(inputs, axis=1, keepdims=True)
# Making dimension (,784,1)
inp1 = keras.ops.expand_dims(inpb, axis=-1)
# Dividing on 784 vectors with dimension (,1,1)
inp1s = keras.ops.split(inp1, INPUTS_SHAPE, axis=1)
# Deleting 1st axis -> result is (,1)
inp2s = [keras.ops.squeeze(inp1s[i], axis=1) for i in range(INPUTS_SHAPE)]
# For each subvector repeating 6 times along 1st axis, 
# Result is 784 subvectors (keras-tensors) with dimension (,6)
inp3s = [keras.ops.repeat(inp2s[i], INPUT_LAYER_SHAPE, axis=1) for i in range(INPUTS_SHAPE)]
# Doing transformation with radial functions
# Transforming them into RF
# Result is 784 keras-tensors with dimension (,6)
inp16s = [modrf(inp3s[i]) for i in range(INPUTS_SHAPE)]
# Concatenating them into one tensor with dimension (,784*6)
inp161s = keras.ops.concatenate(inp16s, axis=1)
# Dividing on 196 vectors with dimensions (,12)
inp162s = keras.ops.split(inp161s, DENCE_LAYER_NUM, axis=1)
# Passing them to Dence layer and result is 196 tensors with dimension (,1)
inp17s=[Dense(1, kernel_initializer='glorot_uniform', use_bias=None)(inp162s[i]) for i in range(DENCE_LAYER_NUM)]
# Concatenating them into one keras-tensor with dimension (,196)
inp18= keras.ops.concatenate(inp17s, axis=1)
x = layers.Dense(112, activation='relu', kernel_initializer='glorot_uniform')(inp18)
# Dropping out 10% of neurons
x0=Dropout(DROPOUT_RATE)(x)
outputs = layers.Dense(OUTPUTS_NUM, activation='softmax')(x0)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers. Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test))

In [ ]:
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


In [ ]:
plt.style.use('seaborn-v0_8')
f,ax = plt.subplots(1,2,figsize=(15,6))

ax[0].plot(history.history['val_loss'],label='Testing Loss')
ax[0].plot(history.history['loss'],label='Training Loss')
ax[0].set_title('Losses',weight='bold',size='x-large')
ax[0].set_xlabel('Epoch',size='large')
ax[0].set_ylabel('Loss',size='large')
ax[0].legend()

ax[1].plot(history.history['val_accuracy'],label='Testing Accuracy')
ax[1].plot(history.history['accuracy'],label='Training Accuracy')
ax[1].set_title('Accuracies',weight='bold',size='x-large')
ax[1].set_xlabel('Epoch',size='large')
ax[1].set_ylabel('Accuracy',size='large')
ax[1].legend()

plt.show()


In [ ]:
model.summary()